## Deep Learning Project

**1) Περιγράψτε (i) πώς διαχωρίσατε το dataset για τα πειράματα σε training/validation/test sets και
(ii) το parameter setting του NeuMF που χρησιμοποιήσατε
(0,5 μονάδα)**

**(i) Dataset Splitting Methodology**

The experiment was conducted using the **MovieLens 100K (ml-100k)** dataset. The raw data, contained in the `u.data` file, consists of 100,000 ratings from 943 users for 1682 movies. To prepare this data for the NeuMF model, a custom Python script (`prepare_ml100k.py`) was implemented to automate the data splitting process, strictly adhering to the methodology described in the NCF paper.

The splitting process followed the **leave-one-out evaluation** protocol, which was implemented in these specific steps:

1.  **Data Loading and ID Conversion**: The `u.data` file, which is tab-separated and has no header, was loaded into a pandas DataFrame. A critical preprocessing step was performed: since the original user and item IDs in the file are 1-based (starting from 1), they were converted to be **0-based** (starting from 0) by subtracting 1 from each ID. This is necessary for the model's embedding layers, which expect zero-indexed inputs.

2.  **Training/Test Split**: For each user, all their interactions were sorted chronologically using the provided timestamp. The single most recent interaction for every user was identified and designated as the **test sample**. All other, older interactions for that user were consequently placed into the **training set**.

3.  **Evaluation Set Generation**: The model's performance is evaluated by its ability to rank a list of items. To create a standardized test for this, the test set was augmented. For each user's single positive test item, **99 other items** that the user had *never* rated were randomly sampled from the entire item pool. These serve as the negative samples for evaluation.

This procedure results in a test environment where, for each of the 943 users, the model must rank a list of 100 items (1 item the user actually liked, and 99 they did not interact with). The final output of this process consists of three files used by the program: `ml-100k.train.rating`, `ml-100k.test.rating`, and `ml-100k.test.negative`. No separate validation set was created; instead, performance was evaluated on the test set after each training epoch.

**(ii) NeuMF Parameter Settings**

The NeuMF model was configured and executed with a specific set of hyperparameters. These were set via a combination of command-line arguments and the script's default values. The exact configuration used was as follows:

*   **Optimizer**: Adam
*   **Learning Rate**: 0.001
*   **Batch Size**: 256
*   **Number of Epochs**: 20
*   **Negative Samples (for training)**: 4 (For each positive interaction in a training batch, 4 negative items were dynamically sampled).
*   **Predictive Factors (GMF Embedding Size)**: 8
*   **MLP Layers**: `[64, 32, 16, 8]`. This defines a "tower" architecture for the MLP part, where the initial concatenated embedding is processed through layers of 64, 32, 16, and finally 8 neurons.
*   **Regularization**: No regularization was applied (`reg_mf=0`, `reg_layers=[0,0,0,0]`).


**2) Δείξτε πώς επηρεάζεται το HR@10, μεταβάλλοντας τα MLP layers από 1 έως 3 με βήμα 1 για
NeuMF (i) με pretraining και (ii) χωρίς pretraining
(1 Μονάδα)**